In [3]:
import numpy as np
import pandas as pd
from definitions import *
from tqdm import tqdm
from scipy import stats

In [2]:
users_aggregated_data = pd.read_csv("./aggregated_data.csv", index_col=0)

In [25]:
df_pvalues = pd.DataFrame()

In [21]:
data = [["test_col",[]]]
for col in users_aggregated_data:
    data.append([col,[]])

In [23]:

for col1 in users_aggregated_data:
    data[0][1].append(col1)
    for col2 in users_aggregated_data:
        r,p = stats.pearsonr(users_aggregated_data[col1], users_aggregated_data[col2])
        for i in range(len(data)):
            if(data[i][0] == col2):
                data[i][1].append(p)
    


In [26]:
for dt in data:
    df_pvalues[dt[0]] = np.array(dt[1])

In [28]:
df_pvalues.to_csv("pvalue_tests.csv")

### threshold = 0.005 (sobs falou)

In [106]:
def run_filters(threshold):
    pvalue_threshold = threshold
    ref = []
    dest = []
    val = []
    for index, row in df_pvalues.iterrows():
        for col in df_pvalues:
            if(col!="test_col"):
                if(row[col]>=pvalue_threshold):
                    if(row["test_col"] in dest and col in ref):
                        continue
                    ref.append(row["test_col"])
                    dest.append(col)
                    val.append(row[col])
    signif_values_df = pd.DataFrame()
    signif_values_df["col_a"] = np.array(ref)
    signif_values_df["col_b"] = np.array(dest)
    signif_values_df["pvalue"] = np.array(val)
    signif_values_df.to_csv("significant_pvalues-"+str(pvalue_threshold)+".csv")

In [107]:
ths_values = [0.1,0.3,0.5,0.05,0.005,0.7,0.8,0.9]
for t_val in ths_values:
    run_filters(t_val)

In [130]:
def map_and_sort_results(threshold):
    sig_df = pd.read_csv("significant_pvalues-"+str(threshold)+".csv",index_col = 0)
    sig_df["col_a"] = sig_df["col_a"].map(lambda col: col.split("_")[1])
    sig_df = sig_df.sort_values(by = ['col_a', 'col_b'], ascending = [True, False], na_position = 'first')
    sig_df.to_csv("significant_pvalues-"+str(threshold)+"-sorted.csv")
    sig_df = sig_df.groupby(['col_a', 'col_b']).count()
    sig_df.to_csv("significant_pvalues-"+str(threshold)+"-count.csv")


In [131]:
map_and_sort_results(0.9)

                   pvalue
col_a col_b              
0     01-00_5           2
      05-00_1           2
      age               2
      asrsa_score       1
      bai_score         1
      sqr_score         2
1     05-00_0           1
      05-01_0           1
      06-00_2           1
      asrsb_score       1
      bai_score         3
      dbi_score         1
      oci_score         2
      sqr_score         3
2     01-00_0           1
      03-00_3           1
      05-00_4           1
      asrsb_score       1
      oci_score         1
3     07-01_2           1
      asrsa_score       2
      bai_score         2
      dbi_score         2
      oci_score         3
      sqr_score         1
4     03-01_0           1
      05-00_0           1
      05-00_3           1
      06-00_2           1
      age               1
      asrsa_score       1
      asrsb_score       2
      bai_score         2
      dbi_score         3
      oci_score         1
      sqr_score         1
5     01-00_

In [127]:
for t_val in ths_values:
    map_and_sort_results(t_val)